<a href="https://colab.research.google.com/github/Alejandro-RL/RNA-Atividade-1.2/blob/main/RNA_Processamento_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento de Dados

# Imports e o arquivo em sí


In [1]:
import pandas as pd
import datetime

In [ ]:
!wget https://www.dropbox.com/s/ys5r54e6a5je0y0/Manaus.csv

In [ ]:
csv = pd.read_csv("/content/Manaus.csv", sep=';')

# Visão Geral dos Casos Confirmados


In [4]:
df = csv[csv['_classificacao'] == 'Confirmado']

# 1) Quantos atributos descrevem cada exemplo? Quais são eles?
(Feito usando o .columns para extrair as colunas, depois o .unique() para ver os valores, que estão em parenteses)

São 39 atributos, que estão listados a seguir:

* Datas de Evolução, Notifição e Sintomas da Doença 
* Classificação (Descartado, Confirmado, Em análise) 
* Conclusão (Recuperado, Óbito, Óbito por outras causas)
* Origem (SIVEP, GAL, eSUS VE) [???]
* Comorbidades, cada uma com uma coluna (Diabetes, Hemato, Hepatica, Imuno, Neurologica, Obessidade, Renal, Respiratoria)
* Critério (Laboratorial, Clínico, Clínico Epidemiológico, Clínico-Imagem)
* Etnia (Etnias Indígenas)
* Evolução (Internado, Internado em UTI, Em tratamento domiciliar)
* Idade
* Faixa Etária (< 1, 1-4, 5-9, 10-14, 15-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80 ou +)
* Gestante
* Profissional de Saúde
* Raça ([Parda, Branca, Preta, Amarela, Indigena, Ignorado)
* Se Notifição (valores numéricos inteiros) [???]
* Sexo (M, F, I)
* Sintomas, cada um com uma coluna (Dispneia, Febre, Garganta,Tosse, Outros)
* SRAG (Síndrome Respiratória Aguda, os valores são os vírus que causam isso, incluindo a covid-19) [???]
* Testes, cada um com uma coluna (Anticorpo, Antígeno, PCR)
* Localização, cada um com uma coluna (Bairro, Bairro_Mapa, Distrito)
* Taxa (valores numéricos fracionados) [???]

In [7]:
df.columns

Index(['_dt_evolucao', '_dt_notificacao', '_dt_sintomas', '_classificacao',
       '_conclusao', '_origem', '_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria', '_criterio', '_etnia',
       '_evolução', '_idade', '_faixa etária', '_gestante', '_profiss_saude',
       '_raca', '_se_notificacao', '_sexo', '_sintoma_dispneia',
       '_sintoma_febre', '_sintoma_garganta', '_sintoma_tosse',
       '_sintoma_outros', '_srag', '_teste_anticorpo', '_teste_antigeno',
       '_teste_pcr', '_bairro', '_bairro_mapa', '_distrito', '_taxa'],
      dtype='object')

In [8]:
len(df.columns)

39



---



# 2) Quantos casos confirmados há em Manaus, cumulativamente?

80420 casos

In [9]:
len(df)

80420



---



#3) A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.

*   Registro mais antigo: 16 de Fevereiro, 2020
*   Registro mais recente: 29 de Dezembro, 2020

In [ ]:
#Extraindo as datas
datas = [i for i in df['_dt_notificacao']]
#Retirando a hora já que são todas iguais
datas = [str(i).strip("00:00") for i in datas]
#datas agora uma lista onde cada elemento é do tipo [dd/mm/aaaa]
datas = [i.split('/') for i in datas]

#Transformando os valores da lista datas no tipo datetime
#Colocando os resultados na lista datas1
datas1 = []
for i in datas:
  #Filtrando as datas vazias
  if("nan" not in i):
    ano = int(i[2])
    mes = int(i[1])
    dia = int(i[0])
    datas1.append(datetime.datetime(ano, mes, dia)) 

#Agora basta usar o sort
datas1 = sorted(datas1)
print("Em mm/dd/aa")
print("Primeira data no registro: "+datas1[0].strftime("%x"))
print("Última data no registro: "+datas1[-1].strftime("%x"))

Em mm/dd/aa
Primeira data no registro: 02/16/20
Última data no registro: 12/29/20




---



# Limpeza dos Dados
* Excluindo todos os atributos relativos
às comorbidades, sintomas, etnia, profissão, outras datas que não a de notificação, origem e outros
que não estiverem envolvidos no contexto do trabalho solicitado.
* Excluindo todas as linhas em que houver dados faltantes para os atributos remanescentes.


In [10]:
#Uma cópia de df com as características relevantes
df1 = df[['_dt_notificacao', '_classificacao','_conclusao', '_idade',
         '_faixa etária', '_sexo','_teste_anticorpo', '_teste_antigeno',
       '_teste_pcr', '_bairro', '_bairro_mapa', '_distrito']]

In [ ]:
#retirando as linhas com os dados faltantes
#No caso das 3 colunas sobre testes, pelo menos 1 terá valor
#Se as 3 estiverem vazias, descarte.
colunas_1 = ['_dt_notificacao', '_classificacao','_conclusao', '_idade',
         '_faixa etária', '_sexo','_bairro', '_bairro_mapa', '_distrito']
for i in colunas_1:
  df1.dropna(subset=[i], inplace=True)

lista_null1 = df1['_teste_anticorpo'].isnull()
lista_null2 = df1['_teste_antigeno'].isnull()
lista_null3 = df1['_teste_pcr'].isnull()

for i in df1.index:
  if(lista_null1[i] and lista_null2[i] and lista_null3[i]):
    df1.drop(index=i, inplace=True)

In [ ]:
#Adicionando uma coluna com datas no formato datetime para usos futuros
datas = [i for i in df1['_dt_notificacao']]
datas = [str(i).strip("00:00") for i in datas]
datas = [i.split('/') for i in datas]

datas1 = []
for i in datas:
  ano = int(i[2])
  mes = int(i[1])
  dia = int(i[0])
  datas1.append(datetime.datetime(ano, mes, dia))

df1['datetime'] = datas1

# 1) Quantos exemplos e atributos há na base de dados após a limpeza e organização?
37.032 Exemplos,
12 Atributos

In [13]:
print(len(df1),len(df1.columns))

37032 13


# 2) Qual a porcentagem de indivíduos recuperados em relação ao todo?
R = 95.5822%

In [14]:
recuperados = df1[df1['_conclusao']=='Recuperado']
porcentagem_recuperados = (len(recuperados)/len(df1))*100
print(porcentagem_recuperados)

95.58219917908836


# 4)Os casos acometeram mais indivíduos do sexo masculino ou feminino?
R = Mais indivíduos do sexo feminino (20302 > 16730)

In [15]:
m = df1[df1['_sexo']=='M']
f = df1[df1['_sexo']=='F']
print(len(m),len(f))

16730 20302


# Baixar o Arquivo já Tratado

In [ ]:
from google.colab import files
df1.to_csv('arquivo_processado.csv')
files.download('arquivo_processado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df1['_bairro_mapa'].unique()

# Coletando bairros com mais casos


In [17]:
casos = {}
for bairro in df1['_bairro_mapa'].unique():
  casos[bairro] = df1[df1['_bairro_mapa'] == bairro].count()[-1]
maiscasos = sorted(casos, key = casos.get, reverse= True)

maiscasos é uma lista que armazena em ordem decrescente as chaves dos bairros com mais casos\
para consultar é só fazer: 


```
#Bairro nº 1 em casos
i = 0
casos[maiscasos[i]]
```



In [18]:
for i in range(3): 
  print(f"{i+1}º: Bairro: "+ str(maiscasos[i])+" possui: " + str(casos[maiscasos[i]]) + " casos")

1º: Bairro: SÃO JOSÉ OPERÁRIO possui: 2192 casos
2º: Bairro: CIDADE NOVA possui: 2017 casos
3º: Bairro: ALVORADA possui: 1840 casos


<h1>4)Qual a média e desvio padrão de idade dos indivíduos que contraíram COVID-19? Qual o indivíduo mais jovem e o mais idoso a contraírem tal enfermidade?</h1>
<p>Media: 42.0787</p>
<p>Desvio padrao: 16.2419</p>

<p>Individuo mais jovem: 0<p/>
<p>Individuo mais idoso = 120 ----- REVISAR (pessoa mais velha viva tem 118 anos</p> 

In [19]:
df1['_idade'].describe()

count    37032.000000
mean        42.078716
std         16.241903
min          0.000000
25%         32.000000
50%         41.000000
75%         52.000000
max        120.000000
Name: _idade, dtype: float64

<h1>5)O bairro com maior incidência de casos</h1>
<p>Bairro: SÃO JOSÉ OPERÁRIO possui: 2192 casos</p>

<h1>6)Os 3 bairros com mais casos confirmados foram:</h1>
<p>1º: Bairro: SÃO JOSÉ OPERÁRIO possui: 2192 casos
<p>2º: Bairro: CIDADE NOVA possui: 2017 casos
<p>3º: Bairro: ALVORADA possui: 1840 casos